#### Simple example of creating a data agent as a tool

In [2]:
import openai
from llama_index.agent import OpenAIAgent
from dotenv import load_dotenv
import os


In [3]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
openai.api_key = api_key

In [5]:
from llama_index.tools.tool_spec.base import BaseToolSpec
import datetime

class DateToolSpec(BaseToolSpec):
    spec_functions = ['get_date']
    
    def get_date(self):
        "A tool to return todays date"
        return datetime.date.today()

In [6]:
example_agent = OpenAIAgent.from_tools(DateToolSpec().to_tool_list(), verbose=True)
print(example_agent.chat("What is the date?"))

=== Calling Function ===
Calling function: get_date with args: {}
Got output: 2023-09-17
Today's date is September 17, 2023.


### Focusing on building an Advanced Search Tool 

this tool will allow agents to search and retrieve information from PubMed

In [15]:
from llama_index.tools.tool_spec.base import BaseToolSpec
import requests
from typing import Optional, List
import xml.etree.ElementTree as ET

SEARCH_URL_TMPL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&usehistory=y&term={term}&retmax={max_results}"


FETCH_URL_TMPL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&&id={ids}"

class PubMedSearchToolSpec(BaseToolSpec):
    
    spec_functions = ['search']
    
    def search(self, term: str, max_results: Optional[int] = 3): 
        response = requests.post(SEARCH_URL_TMPL.format(term=term, max_results=max_results)).json()
        ids = response['esearchresult']['idlist']
        return self.retrieve_documents(ids) 
    
    def retrieve_documents(self, ids: List[str]): 
        response = requests.post(
            FETCH_URL_TMPL.format(ids=",".join(ids))
        ).text
        return response

tool = PubMedSearchToolSpec()
tool.search('nitrogen based mofs')

'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2023//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_230101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="PubMed-not-MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">37712537</PMID><DateRevised><Year>2023</Year><Month>09</Month><Day>15</Day></DateRevised><Article PubModel="Print"><Journal><ISSN IssnType="Electronic">1872-2059</ISSN><JournalIssue CitedMedium="Internet"><Volume>41</Volume><Issue>9</Issue><PubDate><Year>2023</Year><Month>Sep</Month></PubDate></JournalIssue><Title>Se pu = Chinese journal of chromatography</Title><ISOAbbreviation>Se Pu</ISOAbbreviation></Journal><ArticleTitle>[Progress in the application of novel nano-materials to the safety analysis of agricultural products].</ArticleTitle><Pagination><StartPage>731</StartPage><EndPage>741</EndPage><MedlinePgn>731-741</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10

In [1]:
agent = OpenAIAgent.from_tools(PubMedSearchToolSpec().to_tool_list(), verbose=True)

NameError: name 'OpenAIAgent' is not defined

In [ ]:
print(agent.chat('what are the latest developments on nitrogen based drugs in curing cancer?'))